In [1]:
import mss
import sys
import subprocess
import keyboard
import time
import Scripts.boardDecoderMod
import math
import copy
import numpy as np
import chess
import pyautogui
import chess.uci
from skimage.transform import resize
from chess.engine import EngineTerminatedException
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
class BoardReader:
    coords = [0,0,0,0]
    width, height = 0,0
    col = 'w'
    
    def getScreenshot(self):
        mss.mss().shot(mon=1, output='images/screenshot.png')
    
    def getNewBoard(self):    
        self.getScreenshot()
        line = subprocess.check_output(['Scripts/SelectArea.exe','images/screenshot.png']).decode("utf-8")
        line = line[:-2]
        line = line.split(' ')
        for i in range(0,4):self.coords[i] = int(line[i])
        
        if(self.coords[0] > self.coords[2]):self.coords[0],self.coords[2] = self.coords[2],self.coords[0]
        if(self.coords[1] > self.coords[3]):self.coords[1],self.coords[3] = self.coords[3],self.coords[1]

        self.width = abs(self.coords[0] - self.coords[2])
        self.height = abs(self.coords[1] - self.coords[3])
        time.sleep(0.300)
    
    def getFen(self):
        with mss.mss() as sct:
            monitor = {"top": min(self.coords[1],self.coords[3]), 
                       "left": min(self.coords[0],self.coords[2]),
                           "width": self.width, "height": self.height,
                       "mon": 1,}
            sct_img = sct.grab(monitor)
            #Z = zip(sct_img.raw[2::4], sct_img.raw[1::4], sct_img.raw[0::4])
            #pix_array =  np.array([list(a) for a in Z])
            pix_array = np.array(Image.frombytes("RGB", sct_img.size, sct_img.bgra, "raw", "BGRX"))
            start = time.time()
            pix_array = np.dot(pix_array[...,:3], [0.3, 0.59, 0.11]);
            pix_array.resize(self.height,self.width)
            
            if(self.width < self.height):
                pix_array = np.pad(pix_array,((0,0),(int((self.height-self.width)/2),int((self.height-self.width)/2))),'edge')
            elif(self.width > self.height):
                pix_array = np.pad(pix_array,((int((self.width-self.height)/2),int((self.width-self.height)/2)),(0,0)),'edge')
            
            pix_array = resize(pix_array,(224,224),anti_aliasing=True, mode='reflect', preserve_range=True);
                       
            figs = np.zeros((64,28,28))
            k = 0
            for i in range(0,8): 
                for j in range(0,8):
                    figs[k] = pix_array[i*28:(i+1)*28,j*28:(j+1)*28]
                    figs[k] = np.rot90(figs[k])
                    k+=1
            
            f = Scripts.boardDecoderMod.makeFen(board = figs, Col = self.col)
            return f

In [3]:
class Engine:
    fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'
    col = 'w'
    board = chess.Board
    depth = 5
    engine = 0
    def __init__(self, fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1', col = 'w'):
        self.board = chess.Board(fen)
        startBoard = chess.Board('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1')
        A = [a.split() for a in str(startBoard).splitlines()]
        B = [a.split() for a in str(self.board).splitlines()]
        k = 0
        for i in range(0,8):
            for j in range(0,8):
                if(A[i][j] != B[i][j]):
                    k+=1
        if(k <= 2):
            self.board.set_castling_fen('KQkq')
        
        self.fen = fen
        self.col = col
        self.engine = chess.uci.popen_engine('Scripts/stockfish.exe')
        print(self.engine.name)
        print(self.engine.author)
        self.engine.uci()
        self.engine.ucinewgame()
        
    def go(self, fen, coords):
        nBoard = chess.Board(fen)
        print("position: \n",nBoard)
        lm = list(self.board.legal_moves)
        for m in lm:
            self.board.push(m);
            if(str(self.board) == str(nBoard)):
                break
            else: self.board.pop()
        if(str(self.board) != str(nBoard)): self.board = chess.Board(fen)
        
        self.board.turn = chess.WHITE if self.col=='w' else chess.BLACK
        self.engine.position(self.board)
        
        try:
            bMove,ponder = self.engine.go(depth=self.depth)
        except(EngineTerminatedException):
            self.engine = chess.uci.popen_engine('Scripts/stockfish.exe')
            print("restarting engine")
            self.engine.uci()
            self.engine.ucinewgame()
            return
        
        print('best move is: ', bMove)
        
        self.board.push(bMove)
        
        bMove = str(bMove)
        
        
        move = [[coords[0],coords[1]],[coords[0],coords[1]]]
        
        W = int(abs(coords[2] - coords[0])/8);
        H = int(abs(coords[3] - coords[1])/8);
        
        if (self.col == 'w'):
            move[0][0] += (float(ord(bMove[0]) - ord('a')) + 0.5) * W;
            move[0][1] += (7.5 - float(ord(bMove[1]) - ord('1'))) * H;

            move[1][0] += (float(ord(bMove[2]) - ord('a')) + 0.5) * W;
            move[1][1] += (7.5 - float(ord(bMove[3]) - ord('1'))) * H;
        else:
            move[0][0] += (7.5 - float(ord(bMove[0]) - ord('a'))) * W;
            move[0][1] += (float(ord(bMove[1]) - ord('1')) + 0.5) * H;

            move[1][0] += (7.5 - float(ord(bMove[2]) - ord('a'))) * W;
            move[1][1] += (float(ord(bMove[3]) - ord('1')) + 0.5) * H;
        
        pyautogui.click(x = move[0][0], y = move[0][1])
        pyautogui.click(x = move[1][0], y = move[1][1])
                    

In [7]:
def interface():
    reader = BoardReader()
    col = 'none'
    while(1):
        col = 'none'
        print('Press shift+n to select new board\nPress shift+w or shift+b to select your color')
        while(col == 'none'):
            time.sleep(0.05)
            if(keyboard.is_pressed('shift+q')):
                return
            if(keyboard.is_pressed('shift+w')):
                col = 'w'
            if(keyboard.is_pressed('shift+b')):
                col = 'b'
            if(keyboard.is_pressed('shift+n')):
                reader.getNewBoard()
                
        reader.col = col
        engine = Engine(reader.getFen(),col)
        print('return to play move\nend to end game')
        while(True):
            if(keyboard.is_pressed('return')):
                engine.go(reader.getFen(),reader.coords)
            if(keyboard.is_pressed('shift+q')):
                return
            if(keyboard.is_pressed('end')):
                break
            if(keyboard.is_pressed('ctrl+6')):
                while(keyboard.is_pressed('6')):
                    pass
                print('IncreaQsing power', engine.depth)
                engine.depth+=1
                engine.depth = min(25,engine.depth)
            if(keyboard.is_pressed('ctrl+4')):
                while(keyboard.is_pressed('4')):
                    pass
                print('Decreasing power ', engine.depth)
                engine.depth-=1
                engine.depth = max(2,engine.depth)

In [10]:
interface()

Press shift+n to select new board
Press shift+w or shift+b to select your color
